In [92]:
import pandas as pd
from copy import deepcopy
import sys
sys.path.append('/home/ndsviriden/MinMax94/src/utils') 
from Preprocessing import Preprocessor

mypath = '/mnt/HARD/MinMax94/data/data_all/CSV/RP5/'

In [93]:
def parse_datetime_rp5(datetime_rp5):
    date, time = datetime_rp5.split(' ')
    date = '-'.join(date.split('.')[::-1])
    time = time + ':00'
    datetime_standard = date + ' ' + time
    return datetime_standard

## Load RP5 station data from file

In [167]:
wmo_id = 22867
df = pd.read_csv(mypath + str(wmo_id) + '.csv', sep=';', skiprows=6, index_col=False)

columns_to_use = ['T', 'P', 'U', 'DD', 'Ff', 'ff10', 'N', 'VV', 'Td', 'RRR', 'tR', 'W1', 'WW', 'lol']
columns_to_use = [column for column in columns_to_use if column in df.columns]
initial_date_time_column = [column for column in df.columns if column.startswith("Местное время")][0]
columns_to_use.append(initial_date_time_column)
df = df[columns_to_use]

renaming_columns_dict = {initial_date_time_column: 'date_time',
                         'T': 't_air',
                         'P': 'pressure',
                         'U': 'dampness',
                         'DD': 'wind_dir',
                         'Ff': 'wind_velocity',
                         'ff10': 'wind_speedmax',
                         'N': 'cloudiness',
                         'VV': 'visibility',
                         'Td': 'dew_point',
                         'RRR': 'precip_count', # quantity of precipitation (mm)
                         'tR': 'precip_interval', # time interval during which precip_count was calculated
                         'W1': 'precip_type',
                         'WW': 'p_weather'} 

df = df.rename(columns=renaming_columns_dict)
df['date_time'] = pd.to_datetime(df['date_time'].apply(parse_datetime_rp5))

## Wind direction rp5 format convert to mm94 format

In [168]:
converter_wind_dir_dict_rp5 = {
    "Штиль, безветрие": 0,
    "Ветер, дующий с юга": 0,
    "Ветер, дующий с юго-юго-запада": 22.5,
    "Ветер, дующий с юго-запада": 45,
    "Ветер, дующий с западо-юго-запада": 67.5,
    "Ветер, дующий с запада": 90,
    "Ветер, дующий с западо-северо-запада": 112.5,
    "Ветер, дующий с северо-запада": 135,
    "Ветер, дующий с северо-северо-запада": 157.5,
    "Ветер, дующий с севера": 180,
    "Ветер, дующий с северо-северо-востока": 202.5,
    "Ветер, дующий с северо-востока": 225,
    "Ветер, дующий с востоко-северо-востока": 247.5,
    "Ветер, дующий с востока": 270,
    "Ветер, дующий с востоко-юго-востока": 292.5,
    "Ветер, дующий с юго-востока": 315,
    "Ветер, дующий с юго-юго-востока": 337.5
}

wind_dir_values = [value for value in df['wind_dir'].unique() if pd.notnull(value)]

for elem in wind_dir_values:
    print(elem, '-->', str(converter_wind_dir_dict_rp5[elem]) + '°')

Ветер, дующий с запада --> 90°
Ветер, дующий с северо-северо-запада --> 157.5°
Ветер, дующий с западо-юго-запада --> 67.5°
Ветер, дующий с юго-запада --> 45°
Ветер, дующий с юго-юго-запада --> 22.5°
Штиль, безветрие --> 0°
Ветер, дующий с севера --> 180°
Ветер, дующий с северо-запада --> 135°
Ветер, дующий с юга --> 0°
Ветер, дующий с западо-северо-запада --> 112.5°
Ветер, дующий с юго-юго-востока --> 337.5°
Ветер, дующий с юго-востока --> 315°
Ветер, дующий с востоко-юго-востока --> 292.5°
Ветер, дующий с востока --> 270°
Ветер, дующий с востоко-северо-востока --> 247.5°
Ветер, дующий с северо-северо-востока --> 202.5°
Ветер, дующий с северо-востока --> 225°


## Cloudiness rp5 format convert to mm94 format

In [169]:
converter_cloudiness_dict_rp5 = {
    "100%.": 100,
    "90  или более, но не 100%": 90,
    "70 – 80%.": 75,
    "60%.": 60,
    "50%.": 50,
    "40%.": 40,
    "20–30%.": 25,
    "10%  или менее, но не 0": 10,
    "Облаков нет.": 0,
    "Небо не видно из-за тумана и/или других метеорологических явлений.": np.nan
    }

cloudiness_values = [value for value in df['cloudiness'].unique() if pd.notnull(value)]

for elem in cloudiness_values:
    print(elem, '-->', str(converter_cloudiness_dict_rp5[elem]) + '%')

70 – 80%. --> 75%
100%. --> 100%
90  или более, но не 100% --> 90%
20–30%. --> 25%
40%. --> 40%
Облаков нет. --> 0%
50%. --> 50%
10%  или менее, но не 0 --> 10%
60%. --> 60%
Небо не видно из-за тумана и/или других метеорологических явлений. --> nan%


## Precipitation type rp5 format convert to mm94 format

In [170]:
for elem in df['precip_type'].unique():
    print(elem)

Ливень (ливни).
Дождь со снегом или другими видами твердых осадков
Снег и/или другие виды твердых осадков
nan
Дождь.
Облака покрывали более половины неба в течение всего соответствующего периода.
Облака покрывали более половины неба в течение одной части соответствующего периода и половину или менее в течение другой части периода.
Морось.
Облака покрывали половину неба или менее в течение всего соответствующего периода.
Туман или ледяной туман или сильная мгла.
Гроза (грозы) с осадками или без них.
Метель


In [175]:
converter_precip_type_dict_rp5 = {
    np.nan:    0,
    "Облака покрывали более половины неба в течение всего соответствующего периода.": 0,
    "Облака покрывали более половины неба в течение одной части соответствующего периода" + \
    " и половину или менее в течение другой части периода.": 0,
    "Облака покрывали половину неба или менее в течение всего соответствующего периода.": 0,
    "Туман или ледяной туман или сильная мгла.": 0,
    
    "Морось.": 10,
    "Дождь.":  10,
    "Ливень (ливни).": 10,
    "Снег и/или другие виды твердых осадков": 20,
    "Дождь со снегом или другими видами твердых осадков": 30,
    "Метель": 40
    }

precip_type_values = [value for value in df['precip_type'].unique() if pd.notnull(value)]

for elem in precip_type_values:
    try:
        print(elem, '-->', str(converter_precip_type_dict_rp5[elem]))
    except:
        print(elem, '-->', 0)

Ливень (ливни). --> 10
Дождь со снегом или другими видами твердых осадков --> 30
Снег и/или другие виды твердых осадков --> 20
Дождь. --> 10
Облака покрывали более половины неба в течение всего соответствующего периода. --> 0
Облака покрывали более половины неба в течение одной части соответствующего периода и половину или менее в течение другой части периода. --> 0
Морось. --> 10
Облака покрывали половину неба или менее в течение всего соответствующего периода. --> 0
Туман или ледяной туман или сильная мгла. --> 0
Гроза (грозы) с осадками или без них. --> 0
Метель --> 40


## Precipitation count rp5 format convert to mm94 format

In [179]:
converter_precip_count_dict_rp5 = {
    np.nan:    np.nan,
    "Осадков нет": 0,
    "Следы осадков": 0.05
    }